# Number of SNPs

In [ ]:
!pip install -q malariagen_data
!pip install -q scikit-allel

     |████████████████████████████████| 100 kB 3.0 MB/s 
     |████████████████████████████████| 3.7 MB 21.3 MB/s 
     |████████████████████████████████| 301 kB 50.9 MB/s 
     |████████████████████████████████| 9.9 MB 33.6 MB/s 
     |████████████████████████████████| 2.6 MB 23.6 MB/s 
     |████████████████████████████████| 3.6 MB 24.7 MB/s 
     |████████████████████████████████| 185 kB 49.2 MB/s 
     |████████████████████████████████| 5.7 MB 14.6 MB/s 
     |████████████████████████████████| 55 kB 713 kB/s 
     |████████████████████████████████| 1.6 MB 12.1 MB/s 
     |████████████████████████████████| 6.6 MB 7.2 MB/s 


### importing necessary package

In [ ]:
import allel
import malariagen_data
import numpy as np
import pandas as pd
import dask
import dask.array as da
# silence some dask warnings
dask.config.set(**{'array.slicing.split_large_chunks': True})
from dask.diagnostics.progress import ProgressBar

In [ ]:
#import random
from collections import Counter
from tqdm.dask import TqdmCallback
from tqdm.auto import tqdm
import functools


In [ ]:
# plotting setup
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.gridspec import GridSpec
import matplotlib_venn as venn
import seaborn as sns

In [ ]:
#Mounting Google Drive
import os
from google.colab import drive
drive.mount("drive")

# make dir 
results_dir = "drive/MyDrive/"
os.makedirs(results_dir, exist_ok=True)

Mounted at drive


Importing malariagen data set

In [ ]:
ag3 = malariagen_data.Ag3()
#ag3

#samples sets
sets = ["1191-VO-MULTI-OLOUGHLIN-VMF00106", "1191-VO-MULTI-OLOUGHLIN-VMF00140", 
        "AG1000G-BF-A", "AG1000G-BF-B", "AG1000G-BF-C"]
df_samples = ag3.sample_metadata(sample_sets=sets)#.set_index("sample_id")
bf_samples = df_samples.query('country == "Burkina Faso"')
#bf_samples.groupby(['location', 'aim_species']).size()

Load sample metadata:   0%|          | 0/5 [00:00<?, ?it/s]

Let's select the the sample set from 

In [ ]:
from collections import Counter

#Create population column 
bf_samples = df_samples.query('country == "Burkina Faso"')
s1, s2, s3, y,  pop_labels, pop_colors, pop_labels_year, pop_colors_year = [], [], [],[], {}, {}, {}, {}
for iso, species in zip(bf_samples.location, bf_samples.aim_species):
    s1.append(iso[:4]+'_'+species[:3])
bf_samples.insert(4, 'population', s1)
bf_samples_sel = pd.concat([bf_samples, bf_samples])
pop_ids = list(bf_samples.population.unique())

# Pop_year 
for pop, year in zip(bf_samples.population, bf_samples.year):
    y.append(pop+'_'+str(year))
bf_samples.insert(5, 'pop_year', y)
pop_year = list(bf_samples.pop_year.unique())

#pop_id
for idx, species in zip(bf_samples.location, bf_samples.aim_species):
  w = idx.split(' ')[0][:14]
  s2.append(w+' $An.'+species)
val = list(Counter(s2).keys())
for idx, vl in zip(pop_ids, val):
    pop_labels[idx]=vl
pop_labels

for idx, species, year in zip(bf_samples.location, bf_samples.aim_species, bf_samples.year):
  w = idx.split(' ')[0][:14]
  s3.append(w+' $An.'+species+' ('+f'{year}' +')')
val2 = list(Counter(s3).keys())
for idx, vl in zip(pop_year, val2):
    pop_labels_year[idx]=vl
pop_labels_year

#pop_labels
colored = sns.color_palette("husl", len(pop_ids))
for i in range(len(pop_ids)):
    pop_colors[pop_ids[i]] = colored[i]

colored2 = sns.color_palette("husl", len(pop_year))
for i in range(len(pop_year)):
    pop_colors_year[pop_year[i]] = colored2[i]

#pop_cohort
coh_pop = dict([(f"{p}", list(df.index)) for (p), df in bf_samples.reset_index().groupby(['population'])])

#remove some populations 
pop_rm = ['Bana_ara', 'Bana_int', 'Sour_ara']
del s1, s2, s3, y, val, val2, colored, colored2

In [ ]:
query_year = {'Bana_ara_2014': "location == ['Bana Market', 'Bana Village'] and aim_species == 'arabiensis' and year==2014",
              'Bana_col_2012': "location == ['Bana Market', 'Bana Village'] and aim_species == 'coluzzii' and year==2012",
              'Bana_col_2014': "location == ['Bana Market', 'Bana Village'] and aim_species == 'coluzzii' and year==2014",
              'Bana_col_2015': "location == ['Bana Market', 'Bana Village'] and aim_species == 'coluzzii' and year==2015",
              'Bana_col_2016': "location == ['Bana Market', 'Bana Village'] and aim_species == 'coluzzii' and year==2016",
              'Bana_col_2017': "location == ['Bana Market', 'Bana Village'] and aim_species == 'coluzzii' and year==2012",
              'Bana_gam_2012': "location == ['Bana Market', 'Bana Village'] and aim_species == 'gambiae' and year==2012",
              'Bana_gam_2014': "location == ['Bana Market', 'Bana Village'] and aim_species == 'gambiae' and year==2014",
              'Bana_gam_2015': "location == ['Bana Market', 'Bana Village'] and aim_species == 'gambiae' and year==2015", 
              'Bana_gam_2016': "location == ['Bana Market', 'Bana Village'] and aim_species == 'gambiae' and year==2016",
              'Bana_int_2012': "location == ['Bana Market', 'Bana Village'] and aim_species == 'intermediate_gambiae_coluzzii' and year==2012",
              'Mono_gam_2004': "location == 'Monomtenga' and aim_species == 'gambiae' and year==2004",
              'Pala_ara_2014': "location == 'Pala' and aim_species == 'arabiensis' and year==2014",
              'Pala_ara_2015': "location == 'Pala' and aim_species == 'arabiensis' and year==2015",
              'Pala_ara_2016': "location == 'Pala' and aim_species == 'arabiensis' and year==2016",
              'Pala_col_2012': "location == 'Pala' and aim_species == 'coluzzii' and year==2012",
              'Pala_gam_2012': "location == 'Pala' and aim_species == 'gambiae' and year==2012",
              'Pala_gam_2014': "location == 'Pala' and aim_species == 'gambiae' and year==2014",
              'Pala_gam_2015': "location == 'Pala' and aim_species == 'gambiae' and year==2015",
              'Pala_gam_2016': "location == 'Pala' and aim_species == 'gambiae' and year==2016",
              'Pala_gam_2017': "location == 'Pala' and aim_species == 'gambiae' and year==2017",
              'Pala_int_2017': "location == 'Pala' and aim_species == 'intermediate_gambiae_coluzzii' and year==2017",
              'Sour_ara_2017': "location == 'Souroukoudinga' and aim_species == 'arabiensis' and year==2017",
              'Sour_col_2012': "location == 'Souroukoudinga' and aim_species == 'coluzzii' and year==2012",
              'Sour_col_2014': "location == 'Souroukoudinga' and aim_species == 'coluzzii' and year==2014",
              'Sour_col_2015': "location == 'Souroukoudinga' and aim_species == 'coluzzii' and year==2015",
              'Sour_col_2016': "location == 'Souroukoudinga' and aim_species == 'coluzzii' and year==2016",
              'Sour_col_2017': "location == 'Souroukoudinga' and aim_species == 'coluzzii' and year==2017",
              'Sour_gam_2012': "location == 'Souroukoudinga' and aim_species == 'gambiae' and year==2012",
              'Sour_gam_2014': "location == 'Souroukoudinga' and aim_species == 'gambiae' and year==2014",
              'Sour_gam_2015': "location == 'Souroukoudinga' and aim_species == 'gambiae' and year==2015",
              'Sour_gam_2016': "location == 'Souroukoudinga' and aim_species == 'gambiae' and year==2016"}
              
query_sp = {'Bana_ara': "location == ['Bana Market', 'Bana Village'] and aim_species == 'arabiensis'",
            'Bana_col': "location == ['Bana Market', 'Bana Village'] and aim_species == 'coluzzii'",
            'Bana_gam': "location == ['Bana Market', 'Bana Village'] and aim_species == 'gambiae'",
            'Bana_int': "location == ['Bana Market', 'Bana Village'] and aim_species == 'intermediate_gambiae_coluzzii'",
            'Mono_gam': "location == 'Monomtenga' and aim_species == 'gambiae'",
            'Pala_ara': "location == 'Pala' and aim_species == 'arabiensis'",
            'Pala_col': "location == 'Pala' and aim_species == 'coluzzii'",
            'Pala_gam': "location == 'Pala' and aim_species == 'gambiae'",
            'Pala_int': "location == 'Pala' and aim_species == 'intermediate_gambiae_coluzzii'",
            'Sour_ara': "location == 'Souroukoudinga' and aim_species == 'arabiensis'",
            'Sour_col': "location == 'Souroukoudinga' and aim_species == 'coluzzii'",
            'Sour_gam': "location == 'Souroukoudinga' and aim_species == 'gambiae'"
            }

pops1 = ['Bana_col_2012','Bana_col_2014','Bana_col_2015','Bana_col_2016','Bana_col_2017',
       'Bana_gam_2012','Bana_gam_2014','Bana_gam_2015','Bana_gam_2016','Mono_gam_2004',
       'Pala_ara_2014','Pala_ara_2015','Pala_ara_2016','Pala_col_2012','Pala_gam_2012',
       'Pala_gam_2014','Pala_gam_2015','Pala_gam_2016','Pala_gam_2017','Sour_col_2012',
       'Sour_col_2014','Sour_col_2015','Sour_col_2016','Sour_col_2017','Sour_gam_2012',
       'Sour_gam_2014','Sour_gam_2015','Sour_gam_2016']

pops = ['Bana_col', 'Bana_gam', 'Mono_gam', 'Pala_ara', 'Pala_col', 'Pala_gam', 'Sour_col', 'Sour_gam']

In [ ]:
@functools.lru_cache(maxsize=None)
def compute_stats(contig, sample_query, sample_sets=sets):
  # access genotypes
  ds_geno = ag3.snp_calls(region=contig, sample_query=sample_query, sample_sets=sample_sets)
  #count alleles and load into memory
  filt = 'gamb_colu_arab' 
  filt_val = ds_geno[f"variant_filter_pass_{filt}"].values 
  pos = ds_geno["variant_position"][filt_val].values
  gt_filtered = allel.GenotypeDaskArray(ds_geno["call_genotype"][filt_val].data)
    
  # count alleles and load into memory
  with TqdmCallback(desc="Count max alleles"):
    ac = gt_filtered.count_alleles(max_allele=3).compute()
  # compute SNPs  
  n_seg = ac.count_segregating() 
  n_var = ac.count_variant()
  n_biallelic = ac.is_biallelic().sum()
  n_multiallelic = ac.count_segregating() - ac.is_biallelic().sum()
  tx_multiallelic = 100*(ac.count_segregating() - ac.is_biallelic().sum())/ac.count_segregating()
  
  # diversity stats
  nuc_div = allel.sequence_diversity(pos, ac)
  tajima_d = allel.tajima_d(ac, pos)
  w_theta = allel.watterson_theta(pos, ac)

  # list of stats 
  SNPs_list = [n_seg, n_var, n_biallelic, n_multiallelic, tx_multiallelic]
  div_stat = [nuc_div, tajima_d, w_theta]

  return pos, SNPs_list, div_stat


In [ ]:
## compute div stats by pop - village - year
pos_, div, nb_snps = {}, {}, {}
for key in pops1:
  pos_[key], nb_snps[key], div[key] = compute_stats(contig='2R', sample_query=f'{query_year[key]}')

In [ ]:
## compute div stats by pop - village
#pos_X1, div1, nb_snps1 = {}, {}, {}
#for key in pops:
#  pos_X1[key], div1[key], nb_snps1[key] = compute_sfs(contig='X', sample_query=f'{query_sp[key]}')

In [ ]:
## Save data snps 
np.save('drive/MyDrive/Genomic/diversity_stats/2R_nb_snps.npy', nb_snps)
np.save('drive/MyDrive/Genomic/diversity_stats/2R_div.npy', div)


In [ ]:
## load div stats data 
nb_snps_load = np.load('drive/MyDrive/Genomic/diversity_stats/2R_nb_snps.npy', allow_pickle='TRUE').item()
div_load = np.load('drive/MyDrive/Genomic/diversity_stats/2R_div.npy', allow_pickle='TRUE').item()